In [1]:
%%init_spark
launcher.packages = ["harsha2010:magellan:1.0.5-s_2.11"]
launcher.conf.set("spark.ui.port", "4041")
launcher.conf.set("spark.ui.reverseProxyUrl", "https://gisui.totosearch.org")

In [2]:
import org.apache.spark.sql.SQLContext
val sqlCtx = new org.apache.spark.sql.SQLContext(sc)

Intitializing Scala interpreter ...

Spark Web UI available at http://172.17.0.5:4042
SparkContext available as 'sc' (version = 2.2.3, master = local[*], app id = local-1554325728084)
SparkSession available as 'spark'


import org.apache.spark.sql.SQLContext
sqlCtx: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@26315fd7


In [3]:
import magellan.{Point, Polygon}
import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.types._
import spark.implicits._

import magellan.{Point, Polygon}
import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.types._
import spark.implicits._


#### Read the shapefile containing continents

In [4]:
val df = sqlCtx.read.format("magellan").load("/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/")
val polygons = df.select("polygon")

polygons.show()

+--------------------+
|             polygon|
+--------------------+
|magellan.Polygon@...|
|magellan.Polygon@...|
|magellan.Polygon@...|
|magellan.Polygon@...|
|magellan.Polygon@...|
|magellan.Polygon@...|
|magellan.Polygon@...|
|magellan.Polygon@...|
|magellan.Polygon@...|
|magellan.Polygon@...|
+--------------------+



df: org.apache.spark.sql.DataFrame = [point: point, polyline: polyline ... 3 more fields]
polygons: org.apache.spark.sql.DataFrame = [polygon: polygon]


In [5]:
def extractDouble(expectedNumber: Any): Option[Double] = expectedNumber match {
    case d: Double => Some(d)
    case i: Int => Some(i.toDouble)
    case l: Long => Some(l.toDouble)
    case s: String => scala.util.Try(s.trim.toDouble).toOption
    case _ => None
  } 

for (i <- 1 to 100){
    var df_points = spark.read.
    format("csv").
    option("header", "true").
    load(s"/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Partial_grids/Grid_$i")
    
    var grid = df_points.collect()

    var points_grid = sc.parallelize(grid.map(x => (extractDouble(x(1)),extractDouble(x(0)))).collect{case (Some(i), Some(j)) => (i,j)}.toSeq).toDF("Longitude", "Latitude").select(point($"Longitude", $"Latitude").as("point"))

    var cont_points = points_grid.join(polygons).
    where($"point" within $"polygon").
    select("point").collect().
    map(x => x(0).toString).
    map(x => x.split('(')(1).split(')')(0).split(',')).
    map(y => (y(0).toDouble, y(1).toDouble)).toSeq.
    toDF("Longitude", "Latitude").
    write.format("csv").save(s"/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Partial_grids/Grid_$i/grid")

}

extractDouble: (expectedNumber: Any)Option[Double]
